# NYC Apartment Search by Group 46

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_


Imagine that your apartment lease is nearing its end, and it's time to find  a new home in the heart of New York City! To guide us in this quest, we rely on a prudent budget, a preference for a serene neighborhood, and a desire for a touch of greenery. Leveraging the NYC Open Data, including 311 complaints, tree census, and Zillow's historic monthly rent averages, we embark on a data-driven exploration. Let's get started!

## Setup

We begin by importing the necessary libraries and modules. These include tools for data manipulation, visualization, and database interactions. Also, our project relies on a PostgreSQL database for storing and retrieving data. Below are the configuration details. We specify the locations for data files, such as shapefiles and CSVs, as well as constants like API tokens and base URLs for accessing external data sources.

In [1]:
# All import statements needed for the project, for example:

import json
import os
import pathlib
import subprocess
import urllib.parse
from math import ceil
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import shapely
import sqlalchemy as db
from geopy.distance import geodesic
from shapely import wkt
from shapely.geometry import Point, Polygon, mapping

# SQLAlchemy imports for database interaction
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, MetaData, Table, text, func
from sqlalchemy.ext.hybrid import hybrid_property
from sqlalchemy.orm import Session, sessionmaker, declarative_base, column_property
from geoalchemy2 import Geometry, WKTElement, functions as geo_func

# GeoAlchemy2 extensions for geospatial data in SQLAlchemy
from geoalchemy2.functions import ST_Point, ST_Distance

from sqlalchemy.dialects.postgresql import insert
from geoalchemy2 import WKTElement


from math import ceil
import pandas as pd
import geopandas as gpd
import requests
from pathlib import Path


from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import psycopg2
from shapely import wkb

In [2]:
# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "egpaU4U1YY3mBGHMmdNqtmvpv"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "group46project"
DB_USER = "postgres"
DB_URL = f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"

# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

##add new one
# Create the data directory if it doesn't exist
DATA_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In this stage, we take raw data and transform it into a clean and usable format. This process is important for making informed decisions about our future apartment because only with cleaned and organized data, can we analyze and visualize it. We are handling the following data:

**Zipcode Data:** We leverage data on zipcodes, ensuring that each area is accurately represented. This includes removing unnecessary information and aligning the data to a standardized coordinate system.

**311 Complaints Data:** We use the information provided by 311 complaints. By focusing on relevant details like the type of complaints and their locations, we gain insights into the quality of life in each area and make informed decisions about the desirability of potential neighborhoods for our new apartment.

**Tree Data:** We explore the distribution of trees across neighborhoods, looking at factors like health and species diversity. By organizing information of these trees and their location (zipcodes, latitude, and longitude), we gain valuable insights into the green landscape of New York City.

**Zillow's Rent Data:** We clean and organize this data to understand the rental prices over time, helping us make financially sound decisions. Through these steps, we ensure our data is accurate, complete, and ready for finding the apartment in the New York City.


In [4]:
def load_and_clean_zipcodes(zipcode_datafile):
    """
    Load and clean zipcode data from a shapefile.

    This function reads a shapefile containing zipcode data, normalizes its coordinate
    reference system to EPSG 4326 for consistency, retains only relevant columns,
    and converts the zipcode to an integer format.

    Parameters:
    - `zipcode_datafile` (pathlib.Path): The file path to the shapefile containing zipcode data.

    Returns:
    - `gdf_cleaned` (geopandas.GeoDataFrame): A cleaned GeoDataFrame containing zipcode data.
      The DataFrame includes columns 'zipcode' and 'geometry', with zipcode converted to integer.

    Raises:
    - `RuntimeError`: If an unexpected error occurs during the process.

    """
    
    try:
        # Load the shapefile into a GeoDataFrame
        gdf = gpd.read_file(zipcode_datafile)
        
        # Normalize to EPSG 4326 coordinate system for consistency across datasets
        gdf_crs_normalized = gdf.to_crs(epsg=4326)
        
        # Select and rename relevant columns for further analysis
        gdf_cleaned = gdf_crs_normalized[["ZIPCODE", "geometry"]].copy()
        gdf_cleaned.columns = ["zipcode", "geometry"]

        # Convert zipcode to integer for uniform data type
        gdf_cleaned['zipcode'] = gdf_cleaned['zipcode'].astype(int)
    except Exception as e:
        # General exception for unforeseen errors
        raise RuntimeError(f"An unexpected error occurred: {e}")

    return gdf_cleaned

In [5]:
def download_and_clean_311_data(start_date='2015-01-01', end_date='2023-10-01', chunk_size=29999999):
    """
    Download and clean 311 data from New York City for a specified date range.

    This function downloads 311 service requests data from a specified start date to an end date, 
    cleans the data by dropping missing values and converting data types, and then transforms 
    it into a geopandas GeoDataFrame.

    Parameters:
    - `start_date` (str): The start date of the data range. Defaults to '2015-01-01'.
    - `end_date` (str): The end date of the data range. Defaults to '2023-10-01'.
    - `chunk_size` (int, optional): The size of each data chunk to be retrieved. Defaults to 29999999.

    Returns:
    - `geodf_311_data` (geopandas.GeoDataFrame): A GeoDataFrame containing cleaned 311 data, with columns including
      'created_date', 'complaint_type', 'zipcode', and 'geometry'.
      
    Note:
    The 311 data is obtained from the New York City open data API. The resulting GeoDataFrame is saved 
    as a CSV file named '311_DATA.csv' in the 'data' directory.
    """
    
    url = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
    full_data = pd.DataFrame()

    # Calculate the number of chunks needed based on the date range and chunk size
    num_chunks = ceil((pd.to_datetime(end_date) - pd.to_datetime(start_date)).days / chunk_size)

    for chunk in range(num_chunks):
        offset = chunk * chunk_size
        date_filter = f"created_date between '{start_date}' and '{end_date}'"
        params = {
            '$select': 'created_date, complaint_type, incident_zip, latitude, longitude',
            '$where': date_filter,
            '$limit': chunk_size,
            '$offset': offset
        }
        headers = {'X-App-Token': NYC_DATA_APP_TOKEN}
        
        #request data
        response = requests.get(url, headers=headers, params=params)

        data_chunk = pd.DataFrame(response.json())
        # Clean the data: drop missing values and convert data types
        data_chunk_cleaned = data_chunk.dropna(subset=['incident_zip', 'latitude', 'longitude'], how='any').copy()
        data_chunk_cleaned['created_date'] = pd.to_datetime(data_chunk_cleaned['created_date'])
        data_chunk_cleaned['zipcode'] = pd.to_numeric(data_chunk_cleaned['incident_zip'], errors='coerce').dropna().astype(int)
        data_chunk_cleaned = data_chunk_cleaned.drop(columns=['incident_zip'])
        full_data = pd.concat([full_data, data_chunk_cleaned], ignore_index=True)

    # Ensure the directory for data saving exists
    DATA_DIR = Path('data')
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    full_data.to_csv(DATA_DIR / '311_DATA.csv', index=False)
    
    # Convert to GeoDataFrame
    geometry = gpd.points_from_xy(full_data['longitude'], full_data['latitude'])
    geodf_311_data = gpd.GeoDataFrame(full_data, geometry=geometry, crs='EPSG:4326')
    geodf_311_data = geodf_311_data.drop(columns=['longitude', 'latitude'])

    return geodf_311_data

In [6]:
def download_and_clean_tree_data():
    """
    Download and clean tree data from New York City open data.

    This function downloads tree data from the New York City open data API, fills missing values 
    for certain columns, and transforms it into a GeoPandas GeoDataFrame, ready for further analysis.

    Returns:
    - `geodf_tree_data` (geopandas.GeoDataFrame): A GeoDataFrame containing cleaned tree data, 
      with columns including 'created_at', 'tree_id', 'health', 'status', 'spc_common', 
      'zipcode', 'latitude', 'longitude'.

    Raises:
    - `RuntimeError`: If an error occurs during the download process.

    Note:
    The resulting GeoDataFrame is saved as a CSV file named 'TREE_DATA.csv' in the 'data' directory.
 
    """   
    
    url = 'https://data.cityofnewyork.us/resource/5rq2-4hqu.json'
    params = {
        '$select': 'created_at, tree_id, health, status, spc_common, zipcode, latitude, longitude'
    }
    headers = {'X-App-Token': NYC_DATA_APP_TOKEN}

    # Handle network and request errors
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Will raise an HTTPError for unsuccessful status codes
    except requests.exceptions.RequestException as e:
        raise RuntimeError(f"Error downloading data: {e}")

    data = pd.DataFrame(response.json())
    # Fill missing values for specific columns
    columns_to_fillna = ['health', 'status', 'spc_common']
    data[columns_to_fillna] = data[columns_to_fillna].fillna('None')

    # Drop rows with missing zipcode, latitude, or longitude, and copy the dataframe
    data_cleaned = data.dropna(subset=['zipcode', 'latitude', 'longitude'], how='any').copy()

    # Convert 'created_at' to datetime and 'zipcode' to integer, handling errors
    data_cleaned['created_at'] = pd.to_datetime(data_cleaned['created_at'])
    data_cleaned['zipcode'] = pd.to_numeric(data_cleaned['zipcode'], errors='coerce').dropna().astype(int)

    # Ensure the directory for data saving exists
    DATA_DIR = Path('data')
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    data_cleaned.to_csv(DATA_DIR / 'TREE_DATA.csv', index=False)
    
    # Create GeoDataFrame with appropriate CRS
    geometry = gpd.points_from_xy(data_cleaned['longitude'].astype(float), data_cleaned['latitude'].astype(float))
    crs = 'EPSG:4326'
    geodf_tree_data = gpd.GeoDataFrame(data_cleaned, geometry=geometry, crs=crs)
    geodf_tree_data = geodf_tree_data.drop(columns=['longitude', 'latitude'])  # Remove original coordinate columns

    return geodf_tree_data

In [7]:
def load_and_clean_zillow_data():
    """
    Load and clean Zillow rent data for New York City.

    This function loads Zillow rent data, filters for New York City data, cleans and 
    transforms the data into a more usable format for analysis.

    Returns:
    - `zillow_ny` (pandas.DataFrame): Cleaned DataFrame with columns including 'zipcode', 'date', and 'rent'.

    Raises:
    - `FileNotFoundError`: If the file specified by `file_path` is not found.

    Note:
    The Zillow rent data is expected to be stored in a CSV file named 'zillow_rent_data.csv'
    in the 'data' directory. The cleaned data is saved as 'cleaned_zillow_data.csv' in the same directory.
    """
    
    file_path = DATA_DIR / 'zillow_rent_data.csv'

    # Ensure the directory exists before reading the file
    DATA_DIR.mkdir(parents=True, exist_ok=True)

    try:
        df_zillow = pd.read_csv(file_path)
    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {file_path}")

    # Filter data for New York City
    zillow_ny = df_zillow[df_zillow['City'] == 'New York'].copy()

    # Drop unnecessary columns
    columns_to_delete = ['RegionID', 'SizeRank', 'RegionType', 'StateName', 'Metro']
    zillow_ny.drop(columns=columns_to_delete, axis=1, inplace=True)

    # Reset index and rename columns
    zillow_ny.reset_index(drop=True, inplace=True)
    zillow_ny.rename(columns={'RegionName': 'zipcode'}, inplace=True)

    # Replace 0 with NaN and convert column names to lowercase
    zillow_ny.replace(0, np.nan, inplace=True)
    zillow_ny.columns = zillow_ny.columns.str.lower()

    # Reshape the dataframe and convert 'date' to datetime format
    zillow_ny = pd.melt(zillow_ny, id_vars=['zipcode', 'state', 'city', 'countyname'], 
                        var_name='date', value_name='rent')
    zillow_ny['date'] = pd.to_datetime(zillow_ny['date'], errors='coerce', format='%Y-%m-%d')

    # Round 'rent' to 2 decimal places and drop rows with NaN in 'rent'
    zillow_ny['rent'] = zillow_ny['rent'].round(2)
    zillow_ny.dropna(subset=['rent'], inplace=True)

    # Save the cleaned data
    zillow_ny.to_csv(DATA_DIR / 'cleaned_zillow_data.csv', index=False)

    return zillow_ny.drop(columns=['state', 'city', 'countyname'])

In [8]:
def load_all_data():
    """
    Load all datasets that we cleaned.

    Returns:
    - `geodf_zipcode_data` (geopandas.GeoDataFrame): Cleaned GeoDataFrame containing zipcode data.
    - `geodf_311_data` (geopandas.GeoDataFrame): Cleaned GeoDataFrame containing 311 data.
    - `geodf_tree_data` (geopandas.GeoDataFrame): Cleaned GeoDataFrame containing tree data.
    - `df_zillow_data` (pandas.DataFrame): Cleaned DataFrame containing Zillow rent data.
    """
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data()
    geodf_tree_data = download_and_clean_tree_data()
    df_zillow_data = load_and_clean_zillow_data()
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [9]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

In [10]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   zipcode   263 non-null    int32   
 1   geometry  263 non-null    geometry
dtypes: geometry(1), int32(1)
memory usage: 3.2 KB


In [11]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

,zipcode,geometry
0,11436,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."


In [12]:
geodf_311_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 23030863 entries, 0 to 23030862
Data columns (total 4 columns):
 #   Column          Dtype         
---  ------          -----         
 0   created_date    datetime64[ns]
 1   complaint_type  object        
 2   zipcode         float64       
 3   geometry        geometry      
dtypes: datetime64[ns](1), float64(1), geometry(1), object(1)
memory usage: 702.8+ MB


In [13]:
geodf_311_data.head()

,created_date,complaint_type,zipcode,geometry
0,2023-09-30 23:59:58,Noise - Street/Sidewalk,11226.0,POINT (-73.95918 40.65567)
1,2023-09-30 23:59:38,Noise - Residential,11361.0,POINT (-73.78752 40.76676)
2,2023-09-30 23:59:35,Noise - Commercial,10002.0,POINT (-73.98487 40.71950)
3,2023-09-30 23:59:34,Noise - Residential,11435.0,POINT (-73.79729 40.68750)
4,2023-09-30 23:59:28,Noise - Residential,11226.0,POINT (-73.95795 40.65220)


In [14]:
geodf_tree_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   created_at  1000 non-null   datetime64[ns]
 1   tree_id     1000 non-null   object        
 2   health      1000 non-null   object        
 3   status      1000 non-null   object        
 4   spc_common  1000 non-null   object        
 5   zipcode     1000 non-null   int32         
 6   geometry    1000 non-null   geometry      
dtypes: datetime64[ns](1), geometry(1), int32(1), object(4)
memory usage: 50.9+ KB


In [15]:
geodf_tree_data.head()

,created_at,tree_id,health,status,spc_common,zipcode,geometry
0,2015-08-27,180683,Fair,Alive,red maple,11375,POINT (-73.84422 40.72309)
1,2015-09-03,200540,Fair,Alive,pin oak,11357,POINT (-73.81868 40.79411)
2,2015-09-05,204026,Good,Alive,honeylocust,11211,POINT (-73.93661 40.71758)
3,2015-09-05,204337,Good,Alive,honeylocust,11211,POINT (-73.93446 40.71354)
4,2015-08-30,189565,Good,Alive,American linden,11215,POINT (-73.97598 40.66678)


In [16]:
df_zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9039 entries, 5 to 15224
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   zipcode  9039 non-null   int64         
 1   date     9039 non-null   datetime64[ns]
 2   rent     9039 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 282.5 KB


In [17]:
df_zillow_data.head()

,zipcode,date,rent
5,11226,2015-01-31,1944.61
7,10025,2015-01-31,3068.95
13,11206,2015-01-31,2482.83
14,11221,2015-01-31,2125.74
20,11235,2015-01-31,1687.79


## Part 2: Storing Data

In this phase, we transition from the four datasets that we cleaned from Part one to the PostgreSQL database, creating a foundation for seamless data querying and analysis. 

First, we need to create a database to store data!

In [18]:
def setup_new_postgis_database(username, db_name):
    """
    Set up a new PostgreSQL database with the PostGIS extension.

    Args:
    - username (str): The PostgreSQL username.
    - db_name (str): The name for the new database.

    """
    # Create a new database
    subprocess.run(['createdb', db_name])

    # Enable PostGIS extension
    subprocess.run(['psql', '-U', username, '--dbname', db_name, '-c', 'CREATE EXTENSION postgis;'])

In [19]:
setup_new_postgis_database(DB_USER, DB_NAME)

### Creating Tables

After creating the datebase, we can now create tables for the datasets we gained from Part one, which is pretty much like a virtual spreadsheet that organizes specific types of information.

For example, we have a 'zipcodes' table to store details about different zip codes, a 'nyc311s' table to store details about those 311 complaints, a 'trees' table for information about trees, and a 'zillow_datas' table for rental data. 


In [20]:
engine = db.create_engine(DB_URL)

#### Using SQLAlchemy

In [21]:
Base = declarative_base()

class ZipCode(Base):
    """
    Represents the ZipCode data with geometry as POLYGON.
    """
    __tablename__ = 'zipcodes'

    id = Column(Integer, primary_key=True)
    zipcode = Column(Integer, index=True)  # Added index for faster query performance
    geometry = Column(Geometry('POLYGON'))
    
class NYC311(Base):
    """
    Represents the NYC 311 service request data with geometry as POINT.
    """
    __tablename__ = 'nyc311s'

    id = Column(Integer, primary_key=True)
    created_date = Column(DateTime)
    complaint_type = Column(String)
    zipcode = Column(Integer, index=True)
    geometry = Column(Geometry(geometry_type='POINT', srid=4326))

    
class Tree(Base):
    """
    Represents the Tree data with geometry as POINT.
    """
    __tablename__ = "trees"

    id = Column(Integer, primary_key=True)
    created_at = Column(DateTime)
    tree_id = Column(String)
    health = Column(String)
    status = Column(String)
    spc_common = Column(String)
    zipcode = Column(Integer, index=True)
    geometry = Column(Geometry(geometry_type='POINT', srid=4326))

class ZillowData(Base):
    """
    Represents the Zillow rent data.
    """
    __tablename__ = 'zillow_datas'

    id = Column(Integer, primary_key=True)
    zipcode = Column(Integer, index=True)
    date = Column(DateTime)
    rent = Column(Float)

In [22]:
# Create the schema.sql file
with open(DB_SCHEMA_FILE, 'w') as file:
    for table in Base.metadata.tables.values():
        file.write(f"CREATE TABLE IF NOT EXISTS {table.name} (\n")
        for column in table.columns:
            file.write(f"   {column.name} {column.type},\n")
        file.write(");\n\n")

In [23]:
Base.metadata.create_all(engine)

### Add Data to Database

After setting up our tables, now we can add data to these tables in our database! Basically, we go through every dataset, including zip code data, 311 complaints data, tree data, and Zillow data that we organized in Part One, and store every piece of information in the corresponding tables!


#### Using SQLAlchemy

In [24]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [25]:
# Insert data into the database
zipcodes = []
for index, row in geodf_zipcode_data.iterrows():
    zipcode = ZipCode(
        zipcode=row['zipcode'],
        geometry=f'SRID=4326;{row["geometry"].wkt}'  # Directly use the WKT from the geometry column
    )
    zipcodes.append(zipcode)

try:
    session.add_all(zipcodes)  # Add all zipcode objects at once for efficiency
    session.commit()  # Commit the transaction
except Exception as e:
    session.rollback()  # Rollback in case of any error
    raise e


In [ ]:
# Create a GeoDataFrame with an "id" column
geodf_311_data_geometry = gpd.GeoDataFrame(
    geodf_311_data,
    geometry='geometry',
    crs="EPSG:4326"  # Set the coordinate reference system if not already set
)

# Add an "id" column to the GeoDataFrame
geodf_311_data_geometry['id'] = range(1, len(geodf_311_data_geometry) + 1)

# Convert the GeoDataFrame to the specified table using to_postgis
geodf_311_data_geometry.to_postgis(
    "nyc311s",  # Specify the table name
    engine,
    if_exists="replace",  # Use 'replace' or 'fail' based on your requirements
    index=False,  # Set to True if you want to include the index in the database
    dtype={"geometry": Geometry("POINT", srid=4326)},  # Specify the data type for the geometry column
)

In [ ]:
tree_records = []
for index, row in geodf_tree_data.iterrows():
    tree_record = Tree(
        created_at=row['created_at'],  
        tree_id=row['tree_id'],   
        health=row['health'],  
        status=row['status'],  
        spc_common=row['spc_common'],  
        zipcode=row['zipcode'],   
        geometry=f'SRID=4326;{row["geometry"].wkt}'  # Directly use the WKT from the geometry column
    )
    tree_records.append(tree_record)

try:
    session.add_all(tree_records)  # Add all Tree records at once for efficiency
    session.commit()  # Commit the transaction
except Exception as e:
    session.rollback()  # Rollback in case of any error
    raise e


In [ ]:
zillow_data_records = []
for index, row in df_zillow_data.iterrows():
    zillow_data_record = ZillowData(
        zipcode=row['zipcode'],
        date=row['date'],
        rent=row['rent']
    )
    zillow_data_records.append(zillow_data_record)

try:
    session.add_all(zillow_data_records)  # Add all Zillow data records at once for efficiency
    session.commit()  # Commit the transaction
except Exception as e:
    session.rollback()  # Rollback in case of any error
    raise e

## Part 3: Understanding the Data

After we set up all data, tables, and the database, we can use queries to extract meaningful answers to various aspects of apartments, dividing the analysis into six distinct parts to unravel specific parts of the information.

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    """
    Write a SQL query to a file.

    Parameters:
    - query (str): The SQL query to be written to the file.
    - outfile (str): The file path where the SQL query will be saved.
    """
        
    with open(outfile, 'w') as file:
        file.write(query)

### Query 1 - The Calm Places

We're seeking data on the ten zip codes with the fewest 311 complaints between October 1st, 2022, and September 30th, 2023. The result is presented in two columns, displaying each zip code and its corresponding count of complaints in descending order.


In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "top_10_zipcodes_by_calmness.sql"


# SQL query for Query 1: Count of complaints by zipcode within a specified date range
QUERY_1 = """
SELECT zipcode, COUNT(id)
FROM  nyc311s
WHERE created_date BETWEEN '2022-10-01' AND '2023-09-30'
GROUP BY zipcode
ORDER BY COUNT(id) DESC
"""

# Establish a database connection and execute the SQL query
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    
    # Iterate through the query result and print zipcode and complaint counts
    for row in result:
        print(f"zipcode :{row[0]}, Complain Counts: {row[1]}")

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

### Query 2 - The Greenery Spots

We would like explore the ten zip codes where the tree density is the highest only through the trees table. result should have two columns, 10 rows. The output consists of two columns, with each row representing a zip code and its corresponding total number of trees. The results should be sorted in descending order based on the total number of trees.

In [ ]:
QUERY_2_FILENAME = QUERY_DIR / "top_10_zipcodes_by_trees.sql"


# SQL query for Query 2: Top 10 zip codes with the highest count of trees
QUERY_2 = """
SELECT zipcode, COUNT(id)
FROM  trees
GROUP BY zipcode
ORDER BY COUNT(id) DESC
LIMIT 10
"""


In [ ]:
# Establish a database connection and execute the SQL query
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_2))
    
    # Iterate through the query result and print zipcode and tree counts
    for row in result:
        print(f"Zipcode: {row[0]}, Tree Counts: {row[1]}")

In [ ]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

### Query 3 - Evaluating Rent Feasibility Amidst Greenery

In August 2023, we would like to know the 10 zip codes with the highest tree density and understanding the associated rental costs to assess affordability. The output features two columns and 10 rows, sorted in descending order based on total tree count, with rent information included in the results.

In [ ]:
QUERY_3_FILENAME = QUERY_DIR / "rents_for_top10_greenery_zipcodes.sql"


# SQL query for Query 3: Average rent for the top 10 tree-rich zip codes in August 2023
QUERY_3 = """
SELECT zillow_datas.zipcode, TO_CHAR(AVG(zillow_datas.rent), 'FM9,999,999.00') AS average_rent
FROM zillow_datas
JOIN (
    SELECT trees.zipcode, COUNT(trees.id) AS tree_count
    FROM trees
    GROUP BY trees.zipcode
    ORDER BY COUNT(trees.id) DESC
    LIMIT 10
) AS top_trees_zipcodes ON zillow_datas.zipcode = top_trees_zipcodes.zipcode
WHERE zillow_datas.date >= '2023-08-01' AND zillow_datas.date < '2023-09-01'
GROUP BY zillow_datas.zipcode, top_trees_zipcodes.tree_count
ORDER BY top_trees_zipcodes.tree_count DESC
"""


In [ ]:
# Establish a database connection and execute the SQL query
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_3))
    
    # Iterate through the query result and print zipcode and average rent
    for row in result:
        print(f"Zipcode: {row[0]}, Average Rent: ${row[1]}")

In [ ]:
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

### Query 4 - Correlation between Rent, Trees, and Complaints

In January 2023, we are interested in data on the 5 zip codes with the lowest and highest average rent, along with tree and complaint counts, to explore potential correlations. The result is presented in 4 columns (zip code, average rent, tree count, and complaint count) and consist of 10 rows. This includes five rows with the highest average rent and five rows with the lowest average rent, with the rent figures included in the output.


In [ ]:
QUERY_4_FILENAME = QUERY_DIR / "rents_trees_complaints_analysis.sql"


# SQL query for Query 4: Analysis of rent, complaints, and tree counts for top 5 and bottom 5 zip codes by rent in January 2023
QUERY_4 = """
WITH Rentranking AS (
    SELECT
        zipcode,
        AVG(rent) AS average_rent,
        ROW_NUMBER() OVER (ORDER BY AVG(rent) ASC) AS low_rank,
        ROW_NUMBER() OVER (ORDER BY AVG(rent) DESC) AS high_rank
    FROM
        zillow_datas
    WHERE
        date = '2023-01-31'
    GROUP BY zipcode
),
top5rent AS (
    SELECT zipcode, average_rent
    FROM Rentranking
    WHERE low_rank <= 5 OR high_rank <= 5
),
complaint AS (
    SELECT zipcode, COUNT(*) as complaint_count
    FROM nyc311s
    WHERE created_date >= TIMESTAMP '2023-01-01 00:00:00'
    AND created_date < TIMESTAMP '2023-01-31 23:59:59'
    GROUP BY zipcode
),
tree AS (
    SELECT zipcode, COUNT(*) as tree_count
    FROM trees
    GROUP BY zipcode
)
SELECT 
    top5rent.zipcode, 
    TO_CHAR(top5rent.average_rent, 'FM9,999,999.00') AS rent, 
    COALESCE(complaint.complaint_count, 0) as complaint_count, 
    COALESCE(tree.tree_count, 0) as tree_count
FROM 
    top5rent
LEFT JOIN 
    complaint ON top5rent.zipcode = complaint.zipcode
LEFT JOIN 
    tree ON top5rent.zipcode = tree.zipcode
ORDER BY top5rent.average_rent DESC;
"""

In [ ]:
# Establish a database connection and execute the SQL query
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_4))
    
    # Iterate through the query result and print zipcode, rent, complaint count, and tree count
    for row in result:
        print(f"Zipcode: {row[0]}, Rent: ${row[1]}, Complaint Count: {row[2]}, Tree Count: {row[3]}")

In [ ]:
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

### Query 5 - Another Approach to the Greenery Corners

Recalling the top 10 zipcodes with the most trees from Query 2, we're now merging the trees and zipcode tables to locate trees within each zipcode's area, aiming for the same results, which is presented in two columns and consist of 10 rows. The rows should be sorted in descending order based on the total number of trees in each zip code.


In [ ]:
QUERY_5_FILENAME = QUERY_DIR / "tree_by_zipcode_with_polygon.sql"


# SQL query for Query 5: Top 10 zip codes with the highest tree counts based on spatial join
QUERY_5 = """
WITH TreeCounts AS (
    SELECT zipcodes.zipcode, trees.id
    FROM zipcodes
    LEFT JOIN trees ON ST_Within(trees.geometry, zipcodes.geometry)
)

SELECT zipcode, COUNT(DISTINCT id) AS tree_count
FROM TreeCounts
GROUP BY zipcode
ORDER BY tree_count DESC
LIMIT 10;
"""


In [ ]:
# Establish a database connection and execute the SQL query
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_5))
    
    # Iterate through the query result and print zipcode and tree count
    for row in result:
        print(f"Zipcode: {row[0]}, Tree Count: {row[1]}")

In [ ]:
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

### Query 6 - Nearby Trees within a 0.5 Mile Radius."

We aim to gather information about trees, such as species, health, and status, within a 0.5-mile radius of a specified location. The output is consist of 5 columns (ID, species, health, status, and coordinate location) for each tree in the specified radius.


In [ ]:
QUERY_6_FILENAME = QUERY_DIR / "tree_info_in_certain_half_mile_radius.sql"


pointfive=0.5*1609.34

QUERY_6 = '''
SELECT trees.id, trees.spc_common, trees.health, trees.status, ST_AsText(trees.geometry)
FROM trees
WHERE ST_Distance(trees.geometry, ST_SetSRID(ST_MakePoint(-73.96253174434912, 40.80737875669467), 4326)) <= :pointfive
'''


In [ ]:
with engine.connect() as conn:
    result = conn.execute(text(QUERY_6).bindparams(pointfive=pointfive))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

## Part 4: Visualizing the Data

Finally, in Part 4, we are creating 6 visual plots and charts to to illuminate various facets of life in New York City. These visualizations offer a comprehensive view, encompassing details on NYC rents, greenery, noise levels, and geographical locations.


### Visualization 1 - Daily Trends for Top 3 NYC Complaints (Oct '22 - Sep '23).

Firstly, we focus on the daily occurrences for the top 3 complaint types from October 2022 to September 2023, offering a detailed look at their frequency trends throughout this timeframe. This line chart displays the daily complaint trends.


In [ ]:
# Ensure using the 'notebook' backend for Jupyter which supports interactive plots
%matplotlib notebook

def plot_daily_complaint_trends(df):
    """
    Create an animated line plot of complaints over time for different complaint types.

    Parameters:
    - df (DataFrame): Input DataFrame containing information about complaints over time.
    
    Returns:
    - HTML: Displays the animated plot in the notebook.

    """
        
    # Assuming df is your DataFrame and it's already sorted by 'created_date'
    df['created_date'] = pd.to_datetime(df['created_date'])
    df.sort_values('created_date', inplace=True)
    
    # Initialize the figure and line objects for animation
    fig, ax = plt.subplots(figsize=(10, 6))
    lines = [ax.plot([], [], label=complaint_type)[0] for complaint_type in df['complaint_type'].unique()]

    # Set the title and labels
    ax.set_title('Complaints over Time')
    ax.set_xlabel('Date')
    ax.set_ylabel('Count')

    # Function to initialize the background of the animation
    def init():
        for line in lines:
            line.set_data([], [])
        return lines

    # Function to update the data of each line at each frame
    def animate(i):
        for line, complaint_type in zip(lines, df['complaint_type'].unique()):
            # Filter the DataFrame for the current complaint type and get data up to the current frame
            temp_df = df[df['complaint_type'] == complaint_type].iloc[:i+1]
            line.set_data(temp_df['created_date'], temp_df['count'])
        return lines

    # Set plot limits
    ax.set_xlim(df['created_date'].min(), df['created_date'].max())
    ax.set_ylim(df['count'].min(), df['count'].max())
    
    # Include a legend
    ax.legend()

    # Create the animation object
    anim = FuncAnimation(fig, animate, init_func=init, frames=len(df), interval=200, blit=True)

    # Convert the animation to HTML using to_jshtml and display it
    return HTML(anim.to_jshtml())

In [ ]:
def get_data_for_visual_1(engine):
    # Find the top 3 complaint types for October 1st, 2022, to September 30th, 2023
    top_complaint_types_query = """
    SELECT complaint_type, COUNT(id) AS count
    FROM nyc311s
    WHERE created_date BETWEEN '2022-10-01' AND '2023-09-30'
    GROUP BY complaint_type
    ORDER BY COUNT(id) DESC
    LIMIT 3
    """
    top_complaint_types = pd.read_sql_query(top_complaint_types_query, engine)


    # Get daily counts for the top 3 complaint types
    top_complaints = tuple(top_complaint_types['complaint_type'])
    daily_counts_query = """
    SELECT DATE(created_date) as created_date, complaint_type, COUNT(id) AS count
    FROM nyc311s
    WHERE created_date BETWEEN '2022-10-01' AND '2023-09-30'
    AND complaint_type IN %(complaint_types)s
    GROUP BY DATE(created_date), complaint_type
    ORDER BY created_date, complaint_type
    """
    daily_counts = pd.read_sql_query(daily_counts_query, engine, params={'complaint_types': top_complaints})

    return daily_counts


In [ ]:
some_dataframe = get_data_for_visual_1(engine)
plot_daily_complaint_trends(some_dataframe)

### Visualization 2 - Top 10 Complaints by Type in Zip Code 10027 (Oct '18 - Sep '23).

We are exploring the Top 10 Complaint Types in Zip Code 10027. This bar chart shows the frequency of each complaint type, providing insights into the predominant concerns within this area.

In [ ]:
def plot_top_complaints_numbers(dataframe):
    """
    Plot a bar chart showing the number of complaints by type in a specified zip code.

    Parameters:
    - dataframe (DataFrame): Input DataFrame containing information about complaints.

    Returns:
    - None: Displays a bar chart in the notebook.

    """
        
    figure, axes = plt.subplots(figsize=(15, 8))
    complaint_types = dataframe['complaint_type']
    values = dataframe['count']
    axes.bar(complaint_types, values, color='orange')
    axes.set_title(f"Number of Complaints by Type in Zip Code 10027")
    axes.set_xlabel("Complaint Type")
    axes.set_ylabel("Number of Complaints")
    plt.xticks(rotation=45, ha='right')
    plt.show()


In [ ]:
def get_data_for_visual_2():
    query = """
    SELECT complaint_type, COUNT(id) AS count
    FROM  nyc311s
    WHERE created_date BETWEEN '2018-10-01' AND '2023-09-30'
          AND zipcode = '10027'
    GROUP BY complaint_type
    ORDER BY COUNT(id) DESC
    LIMIT 10
    """
    dataframe = pd.read_sql_query(query, engine)
    return dataframe

In [ ]:
some_dataframe = get_data_for_visual_2()
plot_top_complaints_numbers(some_dataframe)

### Visualization 3 - Correlation between Rent, Trees, and Complaints

We are interested in understanding the relationships between rent prices, the number of trees, and the frequency of complaints in different zip codes from January 1st, 2015, to September 30th, 2023.

- Chart 1: Rent vs. Trees:
Explore how living costs relate to greenery. If bars go up, rent is higher; climbing markers indicate a greener neighborhood.

- Chart 2: Rent vs. Complaints:
Understand the link between rent and complaints. Higher bars denote higher rent, while markers reveal the neighborhood's activity level.


In [ ]:
def plot_rent_trees_complaints(df_merged):
    """
    Plot a dual-axis bar chart showing the relationship between rent, number of trees, and number of complaints by zip code.

    Parameters:
    - df_merged (DataFrame): Input DataFrame containing merged information about rent, tree count, and complaint count.

    Returns:
    - None: Displays a dual-axis bar chart in the notebook.

    """
        
    zip_codes = df_merged["zipcode"].astype(str).tolist()
    rents = df_merged['average_rent']
    tree_counts = df_merged['tree_count']
    complaint_counts = df_merged['complaint_count']

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12), sharex=True)

    ax1.bar(zip_codes, rents, color='lightgreen', label='Rent')
    ax1_twin = ax1.twinx()
    ax1_twin.plot(zip_codes, tree_counts, color='darkgreen', marker='o', label='Trees')
    ax1.set_ylabel('Rent ($)')
    ax1_twin.set_ylabel('Number of Trees')
    ax1.set_title('Rent and Number of Trees by Zip Code')
    ax1.legend(loc='upper left')
    ax1_twin.legend(loc='upper right')


    ax2.bar(zip_codes, rents, color='skyblue', label='Rent')
    ax2_twin = ax2.twinx()
    ax2_twin.plot(zip_codes, complaint_counts, color='blue', marker='o', label='Complaints')
    ax2.set_ylabel('Rent ($)')
    ax2_twin.set_ylabel('Number of Complaints')
    ax2.set_xlabel('Zip Code')
    ax2.set_title('Rent and Number of Complaints by Zip Code')
    ax2.legend(loc='upper left')
    ax2_twin.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

In [ ]:
def get_data_for_visual_3():
    """
    Retrieve data for creating visualizations that show the relationship between rent, tree count, and complaint count by zip code.

    Parameters:
    - engine (sqlalchemy.engine.Engine): SQLAlchemy engine for connecting to the database.

    Returns:
    - DataFrame: Merged DataFrame containing information about rent, tree count, and complaint count by zip code.

    """
        
    query_rent_trees = """
    SELECT
        zillow_datas.zipcode,
        AVG(zillow_datas.rent) AS average_rent,
        COUNT(trees.id) AS tree_count
    FROM
        zillow_datas
    LEFT JOIN
        trees ON zillow_datas.zipcode = trees.zipcode
    WHERE
        zillow_datas.date BETWEEN '2015-01-01' AND '2023-09-30'
    GROUP BY
        zillow_datas.zipcode;
    """

    query_rent_complaints = """
    SELECT
        zillow_datas.zipcode,
        AVG(zillow_datas.rent) AS average_rent,
        COUNT(nyc311s.id) AS complaint_count
    FROM
        zillow_datas
    LEFT JOIN
        nyc311s ON zillow_datas.zipcode = nyc311s.zipcode
    WHERE
        zillow_datas.date BETWEEN '2015-01-01' AND '2023-09-30'
    GROUP BY
        zillow_datas.zipcode;
    """


    df_rent_trees = pd.read_sql_query(query_rent_trees, engine)
    
    df_rent_complaints = pd.read_sql_query(query_rent_complaints, engine)

    df = pd.merge(df_rent_trees, df_rent_complaints, on=['zipcode', 'average_rent'])
    return df

In [ ]:
some_dataframe = get_data_for_visual_3()
plot_rent_trees_complaints(some_dataframe)

### Visualization 4 - Rent Affordability vs. Neighbourhood Issues

We are curious that if it's possible to reduce the the noises and complaints if we can afford the high rent for the apartment. Our boxplot, classifying average rent into $1000 bins, provides insights into the correlation with 311 complaints across different zip codes. It seems true that as rent increases, the boxes become narrower, indicating potentially reduced noise and complaints in higher-rent apartments. 


In [ ]:
def plot_rent_complaints(df):
    """
    Plot a boxplot showing the relationship between binned average rent and the number of 311 complaints by zip code.

    Parameters:
    - df (DataFrame): Input DataFrame containing information about average rent, complaint count, and rent bins.

    Returns:
    - None: Displays a boxplot in the notebook.

    """
        
    df['rent_bin'] = pd.cut(df['avg_rent'], bins=range(0, int(df['avg_rent'].max()) + 1000, 1000), right=False)

    # Plotting
    plt.figure(figsize=(20, 10))
    sns.boxplot(x='rent_bin', y='complaint_count', data=df)
    plt.xticks(rotation=45)
    plt.xlabel('Average Rent ($1000 bins)')
    plt.ylabel('Number of 311 Complaints')
    plt.title('311 Complaints vs. Average Rent by Zip Code')
    plt.show()

In [ ]:
def get_data_for_visual_4():
    """
    Retrieve data for creating visualizations that show the relationship between average rent and the number of 311 complaints by zip code.

    Parameters:
    - engine (sqlalchemy.engine.Engine): SQLAlchemy engine for connecting to the database.

    Returns:
    - DataFrame: DataFrame containing information about average rent and complaint count by zip code.

    """
        
    query = '''
    SELECT
        zillow_datas.zipcode,
        AVG(zillow_datas.rent) AS avg_rent,
        COUNT(nyc311s.id) AS complaint_count
    FROM
        zillow_datas
    JOIN
        nyc311s ON zillow_datas.zipcode = nyc311s.zipcode
    WHERE
        zillow_datas.date BETWEEN '2023-09-01' AND '2023-09-30'
        AND nyc311s.created_date BETWEEN '2022-10-01' AND '2023-09-30'
    GROUP BY
        zillow_datas.zipcode
        '''
    df = pd.read_sql_query(query, engine)
    return df

In [ ]:
some_dataframe = get_data_for_visual_4()
plot_rent_complaints(some_dataframe)

### Visualization 5 - Geospatial Snapshot: Recent 311 Incidents in Immediate Proximity

We are trying to find that recent incidents in the immediate neighborhood. The geospatial plot shows a snapshot of 311 cases reported between January 1, 2023, and September 30, 2023, within a 1-kilometer radius of a central point. The blue markers represent incident locations, revealing the areas of recent concern. The red point marks the reference location for context. 


In [ ]:
def plot_311_cases(df):
    """
    Create a geospatial plot of the coordinates from the DataFrame.

    Parameters:
    - df (DataFrame): Input DataFrame containing geospatial information.

    Returns:
    - None: Displays a geospatial plot in the notebook.
    ```
    """
    # Decode the WKB geometry to shapely Points
    df['geometry'] = df['geometry'].apply(lambda x: wkb.loads(x, hex=True))
    
    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    
    # Plot the data
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf.plot(ax = ax, color = 'blue', markersize=5)
    ref_lat=40.80737875669467
    ref_lon=-73.96253174434912 
    plt.scatter(ref_lon, ref_lat, color='red', s=100, label='Reference Point', zorder=5)
    
    # Set title and labels
    ax.set_title('311 Cases from Jan 1, 2023 to Sep 30, 2023 within 1km radius')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    # Show the plot
    plt.show()

In [ ]:
# Function to query 311 cases
def query_311_cases(latitude=40.80737875669467, longitude=-73.96253174434912, radius=1000):
    """
    Query the database for 311 cases within a specified radius of the given latitude and longitude,
    occurring between January 1st, 2023, and September 30th, 2023.

    Parameters:
    - latitude (float): Latitude of the center point (default is 40.80737875669467).
    - longitude (float): Longitude of the center point (default is -73.96253174434912).
    - radius (int): Radius in meters (default is 1000 meters).

    Returns:
    - DataFrame: DataFrame containing information about 311 cases.

    """
    
    # SQL query string
    sql_query = f"""
    SELECT created_date, complaint_type, zipcode, geometry
    FROM nyc311s
    WHERE created_date BETWEEN '2023-01-01' AND '2023-09-30'
    AND ST_DWithin(
        geometry::geography,
        ST_SetSRID(ST_MakePoint({longitude}, {latitude})::geography, 4326),
        {radius}
    )
    """
    # Execute the query and return the results
    df = pd.read_sql_query(sql_query, engine)
    return df

In [ ]:
 plot_311_cases(query_311_cases(latitude=40.80737875669467, longitude=-73.96253174434912, radius=1000))

### Visualization 6 - Enhancing Greenery: Mapping Trees and New Tree Requests in NYC (Oct 2018 - Sep 2023)

We are investigating that whether there are efforts that were made to enhance greenery in the New York City. This geospatial plot illustrates the locations of existing trees and areas with "New Tree Request" 311 complaints reported from October 1, 2018, to September 30, 2023. The green markers represent the current tree locations, while the red markers indicate places where residents have requested new trees. From the plot, we can observe that only some of the requests have been addressed, meaning that there remains room for improvement in addressing community needs. 


In [ ]:
def plot_geospatial_data(trees_df, complaints_df):
    """
    Plot trees data and 'New Tree Request' complaints on a map.

    Parameters:
    trees_df (DataFrame): DataFrame containing trees data.
    complaints_df (DataFrame): DataFrame containing 'New Tree Request' complaints data.
    """
    # Convert WKB to shapely Points
    trees_df['geometry'] = trees_df['geometry'].apply(wkb.loads, hex=True)
    complaints_df['geometry'] = complaints_df['geometry'].apply(wkb.loads, hex=True)

    # Create GeoDataFrames
    trees_gdf = gpd.GeoDataFrame(trees_df, geometry='geometry')
    complaints_gdf = gpd.GeoDataFrame(complaints_df, geometry='geometry')

    # Plot the data
    fig, ax = plt.subplots(figsize=(12, 12))
    trees_gdf.plot(ax=ax, marker='o', color='green', markersize=5, label='Trees')
    complaints_gdf.plot(ax=ax, marker='x', color='red', markersize=5, label='New Tree Requests')

    # Set title and labels
    ax.set_title('NYC Trees and New Tree Requests')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Add legend
    ax.legend()

    # Show plot
    plt.show()


In [ ]:
def query_geospatial_data():
    """
    Query trees data and 'New Tree Request' complaints from the database.

    Parameters:
    conn: Database connection object.

    Returns:
    trees_df (DataFrame): DataFrame containing trees data.
    complaints_df (DataFrame): DataFrame containing 'New Tree Request' complaints data.
    """
    # SQL query for trees
    trees_query = """
    SELECT id, created_at, geometry
    FROM trees
    """

    # SQL query for 'New Tree Request' complaints
    complaints_query = """
    SELECT id, created_date, geometry
    FROM nyc311s
    WHERE complaint_type = 'New Tree Request'
    AND created_date BETWEEN '2018-10-01' AND '2023-09-30'
    """

    # Execute queries and return dataframes
    trees_df = pd.read_sql_query(trees_query, engine)
    complaints_df = pd.read_sql_query(complaints_query, engine)

    return trees_df, complaints_df


In [ ]:
trees_df, complaints_df=query_geospatial_data()
plot_geospatial_data(trees_df, complaints_df)